<a href="https://colab.research.google.com/github/Rashi-Dwivedi1812/CodeQuery/blob/main/code_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask openai pyngrok


In [ ]:
import os
import json
from flask import Flask, request, jsonify
from pyngrok import ngrok
from openai import OpenAI
import threading
from google.colab import userdata # <-- Import userdata to access secrets

# =========================================================
#  LOADING KEYS FROM SECRETS
# =========================================================
API_KEY = userdata.get('OPENAI_API_KEY')
NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
# =========================================================


# --- OpenAI Client Setup ---
client = None
if not API_KEY:
    print("="*50)
    print("ERROR: 'OPENAI_API_KEY' not found in Colab Secrets.")
    print("Please add it (see Cell 2) and restart the runtime.")
    print("="*50)
else:
    try:
        client = OpenAI(api_key=API_KEY)
        print("OpenAI client initialized successfully.")
    except Exception as e:
        print(f"Error initializing OpenAI client: {e}")

# --- Ngrok Setup ---
if not NGROK_TOKEN:
    print("="*50)
    print("ERROR: 'NGROK_AUTH_TOKEN' not found in Colab Secrets.")
    print("Please add it (see Cell 2) and restart the runtime.")
    print("="*50)
else:
    try:
        ngrok.set_auth_token(NGROK_TOKEN)
        print("Ngrok auth token set successfully.")
    except Exception as e:
        print(f"Error setting ngrok auth token: {e}")

# Initialize Flask app
app = Flask(__name__)

# --- UPDATED SYSTEM PROMPT ---
SYSTEM_PROMPT = """You are an expert code assistant.
Given a prompt, return a JSON object with two keys: "code" and "explanation".
The "code" key should contain a complete, runnable code example. For example, C/C++ or Java code should include a main() function and necessary imports. Do not include markdown like ```.
The "explanation" key should contain a concise, step-by-step explanation of what the code does."""

# --- API Endpoints ---
@app.route('/')
def home():
    return "<h1>Flask Server is running on Colab!</h1>"

@app.route('/generate-code', methods=['POST'])
def generate_code():
    if not client:
        return jsonify({"error": "OpenAI client is not initialized. Check your API key."}), 500

    data = request.json
    user_prompt = data.get('prompt')

    if not user_prompt:
        return jsonify({"error": "No prompt provided"}), 400

    try:
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0,
            response_format={"type": "json_object"}
        )

        response_content = completion.choices[0].message.content
        response_data = json.loads(response_content)
        return jsonify(response_data)

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return jsonify({"error": str(e)}), 500

# --- Function to run the app ---
def run_server():
  app.run(port=5000)

# --- Run the App (Non-Blocking) ---
if __name__ == '__main__':
    # Check if both keys were loaded successfully
    keys_are_set = (client is not None and NGROK_TOKEN)

    if not keys_are_set:
        print("="*50)
        print("Cannot start server. Please fix the secret key errors above.")
        print("="*50)
    else:
        # Kill any old/stuck ngrok tunnels
        try:
            ngrok.kill()
        except Exception as e:
            print(f"No existing ngrok tunnels to kill or error: {e}")

        # Start an ngrok tunnel
        public_url = ngrok.connect(5000)
        print("="*80)
        print(f" * SERVER IS RUNNING! Ngrok tunnel is at: {public_url}")
        print(" * Copy this URL to use in Cell 4 or Postman.")
        print(" * Server is running in the background. You can now run Cell 4.")
        print("="*80)

        # Start the server in a background thread
        thread = threading.Thread(target=run_server)
        thread.daemon = True
        thread.start()

OpenAI client initialized successfully.
Ngrok auth token set successfully.
 * SERVER IS RUNNING! Ngrok tunnel is at: NgrokTunnel: "https://42b467210d39.ngrok-free.app" -> "http://localhost:5000"
 * Copy this URL to use in Cell 4 or Postman.
 * Server is running in the background. You can now run Cell 4.
 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
import requests
import json

# 1. PASTE YOUR NGROK URL (from Cell 3 output) inside the quotes
YOUR_NGROK_URL = "https://42b467210d39.ngrok-free.app"

# --- This code sends the request ---

if YOUR_NGROK_URL == "https://<...>.ngrok-free.app":
    print("="*50)
    print("ERROR: Please paste your ngrok URL into the YOUR_NGROK_URL variable above.")
    print("="*50)
else:
    print("--- Interactive Prompt Client ---")
    print("Type your code prompt and press Enter.")
    print("Type 'exit' to quit.\n")

    while True:
        prompt_text = input("Your Prompt: ")

        if prompt_text.lower() == 'exit':
            print("Exiting client.")
            break

        if not prompt_text:
            continue

        try:
            test_url = f"{YOUR_NGROK_URL}/generate-code"
            payload = {"prompt": prompt_text}

            print("...Sending request to server...")
            response = requests.post(test_url, json=payload)
            print(f"Status Code: {response.status_code}")

            if response.status_code == 200:
                response_data = response.json()

                code_val = response_data.get('code')
                expl_val = response_data.get('explanation')

                # This block handles any nested JSON strings (our previous bug fix)
                try:
                    nested_data = json.loads(code_val)
                    code_val = nested_data.get('code')
                    expl_val = nested_data.get('explanation')
                except (json.JSONDecodeError, TypeError):
                    pass

                # Print the code
                print("\n--- GENERATED CODE ---")
                print(code_val)
                print("------------------------")

                # Print the new explanation
                print("\n--- EXPLANATION ---")
                print(expl_val)
                print("------------------------\n")
            else:
                print("--- SERVER RESPONSE (ERROR) ---")
                print(response.text)
                print("------------------------\n")

        except requests.exceptions.RequestException as e:
            print(f"--- REQUEST FAILED ---")
            print(f"An error occurred: {e}")
            print("Is your ngrok URL correct? Is the server in Cell 3 still running?")
            print("------------------------\n")


--- Interactive Prompt Client ---
Type your code prompt and press Enter.
Type 'exit' to quit.

Your Prompt: c++ code for bubble sort
...Sending request to server...


INFO:werkzeug:127.0.0.1 - - [22/Oct/2025 16:10:57] "POST /generate-code HTTP/1.1" 200 -


Status Code: 200

--- GENERATED CODE ---
void bubbleSort(int arr[], int n) {
    for (int i = 0; i < n-1; i++) {
        for (int j = 0; j < n-i-1; j++) {
            if (arr[j] > arr[j+1]) {
                // swap arr[j] and arr[j+1]
                int temp = arr[j];
                arr[j] = arr[j+1];
                arr[j+1] = temp;
            }
        }
    }
}
------------------------

--- EXPLANATION ---
This code implements the bubble sort algorithm to sort an array of integers in ascending order. The outer loop iterates through the array from the first element to the second-to-last element. The inner loop iterates through the unsorted portion of the array and compares adjacent elements. If the current element is greater than the next element, they are swapped. This process is repeated until the array is fully sorted.
------------------------

Your Prompt: exit
Exiting client.
